## Problem Statement 

The assignment of products to vehicle compartments in order to be transported from sources to depot. <br>
Note: Not a PDP yet.

#### Data 

- $G = (V, A)$
- $V = \{0, 1, .., n\}$ where we have $n$ customers and $0$ is the *depot*.
- Cost / Distance Matrix $C$, where $c_{i, \; j}$ is the cost of arc $i$ --> $j$.
- $P = \{1,..\}$ a set of products.
- $M = \{1,..\}$ the set of Vehicle Compartments.
- $CompartmentCapacities = \{q_{m} |\; m \; in \; M\}$
- $CompartmentCapabilities$ Matrix $MxP$
    - $a_{m,\; p} = 1$ if product $p$ can be assigned to compartment $m$
    - $a_{m,\; p} = 0$ otherwise
- Demands Matrix $D$, where $d_{i, \; p}$ is the demand of customer $i$ for product $p$.

#### Decision Variables

- $x_{i,\; j}$
    - $x_{i,\; j} = 1$ if the vehicle is travelling from $i$ to $j$
    - $x_{i,\; j} = 0$ otherwise
- $y_{p,\; m}$
    - $y_{p,\; m} = 1$ if product $p$ is assigned to compartment $m$
    - $y_{p,\; m} = 0$ otherwise
- $u_{i,\; p, \; m}$ which represents the share of demand $d_{i, \; p}$ that is assigned to compartment $m$.
    - Domain $ : \{0, 1, .., d_{i, \; p}\}$ 

#### Constraints
- Every customer (vertex) is visited once.
    <br>
    <center> $\sum_{j \in V} x_{i, \; j} = 1$    $\; \forall i \in V$ \ $\{0\}$ </center>
    <center> $\sum_{j \in V} x_{j, \; i} = 1$    $\; \forall i \in V$ \ $\{0\}$ </center>
- Subtour elimination constraint (necessary?)
    <br>
    <center> $\sum_{i \in S} \sum_{j \in S} x_{i, \; j} \leq |S|  - 1$ $S \subseteq V$ \ $\{0\},  |S| \geq 2$</center>
   
- Capacity-Demand Constraints 
     - The share of a compartment of all demands is restricted by the compartment capacity.
    <br>
         <center> $\sum_{i \in V} \sum_{p \in P} u_{i, \; p, \; m} \leq q_{m} \; \;$  $\forall m \in M$</center>
    <br> 
     - The demand of every customer is fulfilled by the vehicle.
     <br>
         <center> $\sum_{m \in M} u_{i, \; p, \; m} = d_{i, \; p}$ $\; \; \forall i \in V$ \ $\{0\}, \; \forall p \in P$ </center>
     <br>
- Compartment-Product Constraints
     - A compartment must be assigned to **at most one** product.
     <br> 
         <center> $\sum_{p \in P} y_{p, \; m} \leq 1 \; \; \forall m \in M$ </center><br>
     - A product must be assigned to **at least one** compartment.
     <br> 
         <center> $\sum_{m \in M} y_{p, \; m} \geq 1 \; \; \forall p \in P$ </center><br>
      <br>
     - A compartment can be assigned to a product only if it's capable to store this product.
     <br> 
         <center>$y_{p, \; m} \leq  a_{m, \; p} \; \; \forall p \in P, \; \forall m \in M$ </center>
     <br>
     - The demand assignment is restricted by the assignment of the compartments to products.
     <br>
         <center>$u_{i, \; p, \; m} \leq  y_{p, \; m} * d_{i, \; p} \; \; \forall i \in V, \; \forall p \in P, \; \forall m \in M$ </center>


#### Objectice Function
- Minimize the route cost.
 <br>
     <center>$Min \sum_{i \in V} \sum_{j \in V} c_{i, \; j} * x_{i, \; j} $</center>



In [18]:
from gurobipy import * 
import itertools

In [19]:
import random 
random.seed(0)

#### Data 

- $G = (V, A)$
- $V = \{0, 1, .., n\}$ where we have $n$ customers and $0$ is the *depot*.
- Cost / Distance Matrix $C$, where $c_{i, \; j}$ is the cost of arc $i$ --> $j$.
- $P = \{1,..\}$ a set of products.
- $M = \{1,..\}$ the set of Vehicle Compartments.
- $CompartmentCapacities = \{q_{m} |\; m \; in \; M\}$
- $CompartmentCapabilities$ Matrix $MxP$
    - $a_{m,\; p} = 1$ if product $p$ can be assigned to compartment $m$
    - $a_{m,\; p} = 0$ otherwise
- Demands Matrix $D$, where $d_{i, \; p}$ is the demand of customer $i$ for product $p$.

In [20]:
# Assume we have a depot and nb_customer_locations
nb_customer_locations = 5
V = [i for i in range(nb_customer_locations + 1)]
N = nb_customer_locations + 1

# Assume an arbitrary graph
# Distance from / to depot is 0
random.seed(0)
A = [[random.randint(1, 100) if i != 0 and j != 0 else 0 for i in range(N)] for j in range(N)]
C = A

# Assume 2 product types. A type of 300 length and another of 600.
P = [0, 1]

# Assume we have 6 compartments in the vehicle, each of depth 800.
M = [i for i in range(6)]
Capacities = [800 for i in range(6)]

# Compartments 0, 1, 2 and 3 can accommodate products of length  300 (product 0).
# Compartment 4 and 5 can accommodate products of length 600 (product 1).
Capabilities = [
    [1, 0],[1, 0],[1, 0], [1, 0],
    [0, 1],[0, 1],
]

D = [
    # No demands at the depot
    [0, 0],
    # Customer 1 demands 1 item of product 1.
    [1, 0],
    # Customer 2 demands 2 items of product 1.
    [2, 0],
    # Customer 3 demands 1 item of product 1, 1 item of product 2.
    [1, 1],
    # Customer 4 demands 1 item of product 2.
    [0, 1],
    # Customer 5 demands 1 item of product 1, 2 items of product 2.
    [1, 2],
]

# Assume that each item has the depth of 400 and the demand reflects the depth.
D = [[d_ip * 400 for d_ip in d_i] for d_i in D]

In [21]:
print(f"Number of customer locations {nb_customer_locations}")
print(f"The cost matrix \n{C}")
print(f"Customer demands \n{D}")

Number of customer locations 5
The cost matrix 
[[0, 0, 0, 0, 0, 0], [0, 50, 98, 54, 6, 34], [0, 66, 63, 52, 39, 62], [0, 46, 75, 28, 65, 18], [0, 37, 18, 97, 13, 80], [0, 33, 69, 91, 78, 19]]
Customer demands 
[[0, 0], [400, 0], [800, 0], [400, 400], [0, 400], [400, 800]]


In [22]:
# Declare and initialize the model
model = Model("TSPMC")

#### Decision Variables

- $x_{i,\; j}$
    - $x_{i,\; j} = 1$ if the vehicle is travelling from $i$ to $j$
    - $x_{i,\; j} = 0$ otherwise
- $y_{p,\; m}$
    - $y_{p,\; m} = 1$ if product $p$ is assigned to compartment $m$
    - $y_{p,\; m} = 0$ otherwise
- $u_{i,\; p, \; m}$ which represents the share of demand $d_{i, \; p}$ that is assigned to compartment $m$.
    - Domain $ : \{0, 1, .., d_{i, \; p}\}$ 

In [23]:
x = model.addVars(V, V, vtype=GRB.BINARY, name="x")
y = model.addVars(P, M, vtype=GRB.BINARY, name="y")
u = model.addVars(V, P, M, vtype=GRB.INTEGER, name="u")

In [24]:
# Set the bounds of the u vars to the demands
for i, p, m in u:
    u[i, p, m].setAttr(GRB.Attr.LB, 0)
    u[i, p, m].setAttr(GRB.Attr.UB, D[i][p])

model.update()

### Constraints
#### Constraint 1

- Every customer (vertex) is visited once.
    <br>
    <center> $\sum_{j \in V} x_{i, \; j} = 1$    $\; \forall i \in V$ \ $\{0\}$ </center>
    <center> $\sum_{j \in V} x_{j, \; i} = 1$    $\; \forall i \in V$ \ $\{0\}$ </center>
    <br>


In [25]:
# TODO: check the need to exclude the depot
# If the depot is excluded we, can get a solution the that visits the depot more than once.
# ?? \Resolved: yes we need to exclude the depot, otherwise it appears mid tour.

# Every vertex is left once
model.addConstrs((sum(x[i, j] for j in V) == 1 for i in V), name="out_vertex_constr")

# Every vertex is entered once
model.addConstrs((sum(x[j, i] for j in V) == 1 for i in V), name="in_vertex_constr")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

In [26]:
model.update()
model.getConstrs()

[<gurobi.Constr out_vertex_constr[0]>,
 <gurobi.Constr out_vertex_constr[1]>,
 <gurobi.Constr out_vertex_constr[2]>,
 <gurobi.Constr out_vertex_constr[3]>,
 <gurobi.Constr out_vertex_constr[4]>,
 <gurobi.Constr out_vertex_constr[5]>,
 <gurobi.Constr in_vertex_constr[0]>,
 <gurobi.Constr in_vertex_constr[1]>,
 <gurobi.Constr in_vertex_constr[2]>,
 <gurobi.Constr in_vertex_constr[3]>,
 <gurobi.Constr in_vertex_constr[4]>,
 <gurobi.Constr in_vertex_constr[5]>]

#### Constraint 2

- Subtour elimination constraint (necessary?)
    
    
<center> $\sum_{i \in S} \sum_{j \in S} x_{i, \; j} \leq |S|  - 1$ $S \subseteq V$ \ $\{0\},  |S| \geq 2$</center>
   

In [27]:
def get_possible_subtours(nb_vertices):
    V_ = [i for i in range(nb_vertices)]
    return [list(t) for i in range(2, len(V_)) for t in list(itertools.combinations(V_, i))]

possible_sub_tours = get_possible_subtours(len(V))
for sub_tour in possible_sub_tours:
    model.addConstr(sum(x[i, j] for i in sub_tour for j in sub_tour) <= (len(sub_tour) - 1), name=f"sub_tour_constr_{sub_tour}")

#### Constraint

- Capacity-Demand Constraints 

     - The share of a compartment of all demands is restricted by the compartment capacity.
    <br>
         <center> $\sum_{i \in V} \sum_{p \in P} u_{i, \; p, \; m} \leq q_{m} \; \;$  $\forall m \in M$</center>
    <br> 
     - The demand of every customer is fulfilled by the vehicle.
     <br>
         <center> $\sum_{m \in M} u_{i, \; p, \; m} = d_{i, \; p}$ $\; \; \forall i \in V$ \ $\{0\}, \; \forall p \in P$ </center>
     <br>

In [28]:
cap_constrs = model.addConstrs((sum(u[i, p, m] for i in V for p in P) <= Capacities[m] for m in M), name="capacity_constr")
dem_constrs = model.addConstrs((sum(u[i, p, m] for m in M) == D[i][p] for i in V for p in P), name="demand_constr")

#### Constraint 

- Compartment-Product Constraints
     - A compartment must be assigned to **at most one** product.
     <br> 
         <center> $\sum_{p \in P} y_{p, \; m} \leq 1 \; \; \forall m \in M$ </center><br>
     - A product must be assigned to **at least one** compartment.
     <br> 
         <center> $\sum_{m \in M} y_{p, \; m} \geq 1 \; \; \forall p \in P$ </center><br>
      <br>
     - A compartment can be assigned to a product only if it's capable to store this product.
     <br> 
         <center>$y_{p, \; m} \leq  a_{m, \; p} \; \; \forall p \in P, \; \forall m \in M$ </center>
     <br>
     - The demand asignment is restricted by the assignment of the compartments to products.
     <br>
         <center>$u_{i, \; p, \; m} \leq  y_{p, \; m} * d_{i, \; p} \; \; \forall i \in V, \; \forall p \in P, \; \forall m \in M$ </center>

In [29]:
comp_prod_constrs = model.addConstrs((sum(y[p, m] for p in P) <= 1 for m in M), name="compartment_product_constr")
prod_comp_constrs = model.addConstrs((sum(y[p, m] for m in M) >= 1 for p in P), name="product_compartment_constr")
comp_cap_constrs = model.addConstrs((y[p, m] <= Capabilities[m][p] for p in P for m in M), name="compartment_capability_constr")
share_dem_constrs = model.addConstrs((u[i, p, m] <= y[p, m] * D[i][p] for i in V for p in P for m in M), name="share_demand_constr")

### Objectice Function

- Minimize the route cost.
 <br>
     <center>$Min \sum_{i \in V} \sum_{j \in V} c_{i, \; j} * x_{i, \; j} $</center>

In [30]:
# Set the objective function
model.setObjective((sum(C[i][j] * x[i, j] for i in V for j in V)), GRB.MINIMIZE)

In [31]:
model.write('TSPMC_1.lp')

In [32]:
# Run the optimization engine (implicitly runs model.update())
model.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 178 rows, 120 columns and 996 nonzeros
Model fingerprint: 0x8463d1d4
Variable types: 0 continuous, 120 integer (48 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [6e+00, 1e+02]
  Bounds range     [1e+00, 8e+02]
  RHS range        [1e+00, 8e+02]
Found heuristic solution: objective 291.0000000
Presolve removed 110 rows and 84 columns
Presolve time: 0.00s
Presolved: 68 rows, 36 columns, 702 nonzeros
Variable types: 0 continuous, 36 integer (36 binary)

Root relaxation: objective 7.500000e+01, 14 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      75.0000000   75.00000  0.00%     -    0s

Explored 1 nodes (14 simplex iterati

In [33]:
def print_solution(optimized_model):
    for var in optimized_model.getVars():
        if abs(var.x) > 1e-6:
            print("{0}: {1}".format(var.varName, var.x))
    print("Total cost: {0}".format(optimized_model.objVal))
    return None

# display optimal values of decision variables
print_solution(model)  

x[0,3]: 1.0
x[1,4]: 1.0
x[2,0]: 1.0
x[3,5]: 1.0
x[4,2]: 1.0
x[5,1]: 1.0
y[0,0]: 1.0
y[0,1]: 1.0
y[0,2]: 1.0
y[0,3]: 1.0
y[1,4]: 1.0
y[1,5]: 1.0
u[1,0,1]: 400.0
u[2,0,1]: 400.0
u[2,0,3]: 400.0
u[3,0,3]: 400.0
u[3,1,4]: 400.0
u[4,1,4]: 400.0
u[5,0,0]: 400.0
u[5,1,5]: 800.0
Total cost: 75.0


In [34]:
# The order of visiting the vertices
nexts = [j for i in V for j in V if x[i, j].x == 1]

# We always start at the depot
order = [0] 
curr = nexts[0]

# We always end at the depot
while (curr !=0):
    order.append(curr)
    curr = nexts[curr]
    
print(f"The order of visiting the customers is {order[1:]}\n")

# Which demands are mapped to which compartments
for i in order[1:]:
    loading = [[u[i, p, m].x for p in P] for m in M]
    loading_str = f"Loading at customer {i}:\n"
    loading_str += ", ".join(["comp_{}: {}".format(m, ["prod_{}: {}".format(p, int(loading[m][p])) for p in P if loading[m][p]]) for m in M if sum(loading[m])])
    loading_str += "\n"
    print(loading_str)
    
backpack = {m: [] for m in M}
for m in M:
    loading = [[u[i, p, m].x for p in P] for i in V]
    backpack[m].extend(int(loading[i][p]) for i in V for p in P if int(loading[i][p]))

backpack_str = "Backpack utilization: \n"
backpack_str += ", ".join([f"comp_{m}: {sum(vals)}" for m, vals in backpack.items()])
print(backpack_str)

The order of visiting the customers is [3, 5, 1, 4, 2]

Loading at customer 3:
comp_3: ['prod_0: 400'], comp_4: ['prod_1: 400']

Loading at customer 5:
comp_0: ['prod_0: 400'], comp_5: ['prod_1: 800']

Loading at customer 1:
comp_1: ['prod_0: 400']

Loading at customer 4:
comp_4: ['prod_1: 400']

Loading at customer 2:
comp_1: ['prod_0: 400'], comp_3: ['prod_0: 400']

Backpack utilization: 
comp_0: 400, comp_1: 800, comp_2: 0, comp_3: 800, comp_4: 800, comp_5: 800
